# Kili Tutorial: Importing predictions
In this tutorial, we will show how to import predictions (pre-annotations) into Kili in order to help annotators and accelerate the whole annotation process. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way, but also to dive into the actual process of iteratively developing real applications in Kili.

Additionally:

For an overview of Kili, visit https://kili-technology.com You can also check out the Kili documentation https://cloud.kili-technology.com/docs. Our goal is to export labels that can predict whether an image contains a Porsche or a Tesla.

The tutorial is divided into four parts:

1. Understanding the different types of labels
2. Understanding the data model of a label
3. Pushing predictions to Kili
4. Visualizing predictions in Kili

## 1. Understanding the different types of labels
A label is the annotation or combination of all annotations created on an asset. For example, all houses identified on the satellite image, or all the information annotated text on the document.

There are four categories of labels:

- **default**: an ordinary label, made by an annotator
- **prediction**: a pre-annotation, made by a model
- **autosave**: a temporary label, made by the app every minute while annotating
- **review**: a check, carried out by a reviewer

When you export data (see [How to export labels](https://github.com/kili-technology/kili-playground/blob/master/recipes/export_labels.ipynb)), you can find out which category a label belongs to by looking at the field `labelType`. It can take the following values: `PREDICTION`, `DEFAULT`, `AUTOSAVE`, `REVIEW`.

## 2. Understanding the data model of a label
Predictions are pushed in Kili using Python dictionaries. The format of the dictionary to be pushed depends on the type of data (text, image, audio ...), the machine learning task(s) (e.g. simple, multiple classification, transcription, named entity recognition, object detection, etc ...) and their order. In summary, it depends on the JSON format that describes the interface of your annotation project.

The following cells will show you how to view this JSON. You need to update `api_key` and `api_endpoint` before, or to have set those as global environment variables.

In [ ]:
# Authentication
import os

#!pip install kili # uncomment if you don't have kili installed already
from kili.client import Kili

api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT') # If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

Let us create a new image classification project and try to retrieve the label of one asset, to understand the **data model**.

In [ ]:
title = 'Example'
input_type = 'IMAGE'
json_interface = {
	"jobs": {
		"JOB_0": {
			"mlTask": "CLASSIFICATION",
			"required": 1,
			"content": {
				"categories": {
					"OBJECT_A": {
						"name": "Object A",
						"children": []
					},
					"OBJECT_B": {
						"name": "Object B",
						"children": []
					}
				},
				"input": "radio"
			}
		}
	}
}
project = kili.create_project(input_type=input_type, json_interface=json_interface, title=title)

Then, make sure the project created has assets already uploaded. You can upload some using the `append_many_to_dataset` function as we use in this notebook. To help you, you can also check out the recipe [Create a project](https://github.com/kili-technology/kili-playground/blob/master/recipes/create_project.ipynb).
On Kili platform, label the first asset and execute the following code to retrieve it.

In [ ]:
assets = kili.assets(project_id=project['id'])
if assets:
    label = assets[0]['labels']
    print(label)

Taking the previous example of image classification task, here are the fields you will have for the data model of a label :

- `author`: User that created the Label.
  - `email`: Email of the user
  - `id`: Unique identifier of the User in the Kili database
- `createdAt`: Date of creation of the Label.
- `id`: Unique identifier of the Label in the Kili database.
- `jsonResponse`: List of label annotations. See detail lower.
  - `CLASSIFICATION_JOB`: First annotation task, as defined in the interface builder.
    - `categories`: Category the Asset belongs to.
      - `name`: Name of the category.
      - `confidence`: Category Confidence Index. The value is between 0 and 100 for a PREDICTION type Label (produced by a model). The value is always 100 for a Label created by a human.

For other types of tasks the fields will be the same except for the `jsonResponse` for which we follow more or less the data model of the Google APIs.

## 3. Pushing predictions to Kili
To make a prediction in Kili, you need 4 pieces of information:

- A project ID
- An external asset ID
- A model name (arbitrary)
- A jsonResponse, in the same format as the labels `jsonResponse` (see above)

We are going to present an example for each category of project, with an interface of project, one or two assets and the form of the associated `jsonResponse` for one prediction for one of the assets.

We'll also recall the procedure to get to this point, including the creation of a project, the importation of assets into those projects, and, finally, the importation of predictions. This process will be triggered for every example at the end of the notebook.

In [1]:
project_examples = []

### a) Classification examples

#### - Single-class classification

In [ ]:
project_examples.append(
    {
        'title': 'Spam classification',
        'description': 'Classify text as spam or not',
        'input_type': 'TEXT',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "Is it a spam?",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "YES_IT_IS_SPAM": { "name": "Yes, it is spam", "children": [] },
                            "NO": { "name": "No", "children": []}
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [
            'Lorem ipsum dolor sit amet, consectetur adipiscing elit.',
        ],
        'json_response': {
            "JOB_0": {
                "categories": [{"name": "YES_IT_IS_SPAM", "confidence": 100 }]
            }
        },
        'model_name': 'spam-classification-v0.0.1'
    }
)

#### - Multi-class classification

In [ ]:
project_examples.append(
    {
        'title': 'Delivery issue classification',
        'description': 'Classify the problems encountered by the users',
        'input_type': 'TEXT',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "What was the problem encountered?",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "DELIVERY": { "name": "Delivery", "children": [] },
                            "BAD_PRODUCT": { "name": "Bad product", "children": [] },
                            "OPINION_CHANGED": { "name": "Opinion changed", "children": [] },
                            "ILLEGAL_PRODUCT": { "name": "Illegal product", "children": [] }
                        },
                        "input": "checkbox"
                    }
                }
            }
        },
        'assets_to_import': [
            'Good morning, I wanted to contact you about my delivery, that is not what was supposed to be', 
            'NOT THE GOOD DELIVERY AND LATE'
        ],
        'json_response': {
            "JOB_0": {
                "categories": [
                    { "name": "BAD_PRODUCT", "confidence": 100 },
                    { "name": "ILLEGAL_PRODUCT", "confidence": 100 }
                ]
            }
        },
        'model_name' : 'delivery-problem-classification-v0.0.1'
    }
)

#### - Single-class classification, with 2 different jobs

In [ ]:
project_examples.append(
    {
        'title': 'News articles',
        'description': 'Classify text as a news article and describe the type of language',
        'input_type': 'TEXT',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "Is it a news article?",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "YES_IT_IS_A_NEWS_ARTICLE": { "name": "Yes, it is a news article", "children": [] },
                            "NO": { "name": "No.", "children": [] }
                        },
                        "input": "radio"
                    }
                },
                "JOB_1": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "What type of language is used?",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "FORMAL": { "name": "Formal", "children": [] },
                            "COMMON": { "name": "Common", "children": [] }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [
            'Lorem ipsum dolor sit amet, consectetur adipiscing elit.', 
            'Vestibulum in elit sit amet turpis sagittis venenatis ac in nisl.'
        ],
        'json_response': {
            "JOB_0": {
                "categories": [{ "name": "YES_IT_IS_A_NEWS_ARTICLE", "confidence": 100 }]
            },
            "JOB_1": {
                "categories": [{ "name": "COMMON", "confidence": 100 }]
            }
        },
        'model_name': 'news-article-classification-v0.0.1'
    }
)

#### - Single-class classification with nested classification

In [ ]:
project_examples.append(
    {
        'title': 'News articles about sports',
        'description': 'Classify text as a news article dealing with sports',
        'input_type': 'TEXT',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "Is it a news article?",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "YES_IT_IS_A_NEWS_ARTICLE": {
                                "name": "Yes, it is a news article.",
                                "children": ["NESTED_JOB"]
                            },
                            "NO": {
                                "name": "No.",
                                "children": []
                            }
                        },
                        "input": "radio"
                    }
                },
                "NESTED_JOB": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "What is the article about?",
                    "required": 1,
                    "isChild": True,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "SPORTS": { "name": "Sports", "children": [] },
                            "OTHERS": { "name": "Others", "children": [] }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [
            'Lorem ipsum dolor sit amet, consectetur adipiscing elit.', 
            'Vestibulum in elit sit amet turpis sagittis venenatis ac in nisl.'
        ],
        'json_response': {
            "JOB_0": {
                "categories": [{"name": "YES_IT_IS_A_NEWS_ARTICLE", "confidence": 100 }],
                "children": {
                    "NESTED_JOB": {
                        "categories": [{ "name": "SPORTS", "confidence": 100 }]
                    }
                }
            }
        },
        'model_name': 'news-article-classification-v0.0.2'
    }
)

### b) Object detection examples

#### - Bounding box object

In [ ]:
project_examples.append(
    {
        'title': 'Car brand recognition',
        'description': 'Identify and locate cars',
        'input_type': 'IMAGE',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "rectangle"
                    ],
                    "instruction": "What car brand?",
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "TESLA": { "name": "Tesla", "children": [] },
                            "FERRARI": { "name": "Ferrari", "children": [] }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [
            "https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg", 
            "https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg"
        ],
        'json_response': {
            "JOB_0": {
                "annotations": [{
                    "boundingPoly": [{
                        "normalizedVertices": [
                            { "x": 0.16, "y": 0.82},
                            { "x": 0.16, "y": 0.32 },
                            { "x": 0.82, "y": 0.32 },
                            { "x": 0.82, "y": 0.82 }
                        ]}
                    ],
                    "categories": [{ "name": "TESLA", "confidence": 100 }],
                    "mid": "unique-tesla",
                    "type": "rectangle",
                }]
            }
        },
        'model_name': 'car-brand-localisation-v0.0.1'
    }
)

#### - Bounding box with nested classification

In [ ]:
project_examples.append(
    {
        'title': 'Car brand & color recognition',
        'description': 'Identify brand and color',
        'input_type': 'IMAGE',
        'json_interface': {
            "jobs": {
                "CLASSIFICATION_JOB": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "What color?",
                    "required": 0,
                    "isChild": True,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "RED": { "name": "Red", "children": [] },
                            "BLACK": { "name": "Black", "children": [] },
                            "WHITE": { "name": "White", "children": [] },
                            "GREY": { "name": "Grey", "children": [] }
                        },
                        "input": "checkbox"
                    },
                },
                "JOB": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "rectangle"
                    ],
                    "instruction": "What car brand?",
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "TESLA": {
                                "name": "Tesla",
                                "children": ["CLASSIFICATION_JOB"]
                            },
                            "FERRARI": {
                                "name": "Ferrari",
                                "children": ["CLASSIFICATION_JOB"]
                            }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [
            "https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg",
            "https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg"
        ],
        'json_response': {
            "JOB": {
                "annotations": [{
                    "boundingPoly": [{
                        "normalizedVertices": [
                            { "x": 0.09, "y": 0.84 },
                            { "x": 0.09, "y": 0.36 },
                            { "x": 0.92, "y": 0.36 },
                            { "x": 0.92, "y": 0.84 }
                        ]
                    }],
                    "categories": [{ "name": "FERRARI", "confidence": 100 }],
                    "mid": "unique-ferrari",
                    "type": "rectangle",
                    "children": {
                        "CLASSIFICATION_JOB": {
                            "categories": [{ "name": "GREY", "confidence": 100 }]
                        }
                    }
                }]
            }
        },
        'model_name': 'car-brand-color-v0.0.1'
    }
)

#### - Bounding boxes with relations

In [ ]:
project_examples.append(
    {
        'title': 'Car and wheels recognition',
        'description': 'Identify and locate cars and their wheels',
        'input_type': 'IMAGE',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "rectangle"
                    ],
                    "instruction": "What part of the car can you see?",
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "WHOLE_CAR": { "name": "Whole car", "children": [] },
                            "WHEELS": { "name": "Wheels", "children": [] }
                        },
                        "input": "radio"
                    }
                },
                "RELATION_JOB": {
                    "mlTask": "OBJECT_RELATION",
                    "instruction": "Car relations",
                    "required": 0,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "WHOLE_CAR_AND_WHEELS": {
                                "name": "Whole car and wheels",
                                "children": [],
                                "startObjects": ["WHOLE_CAR"],
                                "endObjects": ["WHEELS"]
                            },
                        },
                        "input": "radio"
                    },
                }
            }
        },
        'assets_to_import': [
            "https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg", 
            "https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg",
        ],
        'json_response': {
            "JOB_0": {
                "annotations": [{
                    "boundingPoly": [{
                        "normalizedVertices": [
                            { "x": 0.16, "y": 0.82},
                            { "x": 0.16, "y": 0.32 },
                            { "x": 0.82, "y": 0.32 },
                            { "x": 0.82, "y": 0.82 }
                        ]}
                    ],
                    "mid": "car",
                    "type": "rectangle",
                    "categories": [{ "name": "WHOLE_CAR", "confidence": 100 }],
                },
                {
                    "boundingPoly": [{
                        "normalizedVertices": [
                            { "x": 0.54, "y": 0.59 },
                            { "x": 0.43, "y": 0.59 },
                            { "x": 0.43, "y": 0.83 },
                            { "x": 0.54, "y": 0.83 }
                        ]}
                    ],
                    "mid": "left-front-wheel",
                    "type": "rectangle",
                    "categories": [{ "name": "WHEELS", "confidence": 100 }],
                },
                {
                    "boundingPoly": [{
                        "normalizedVertices": [
                            { "x": 0.81, "y": 0.57},
                            { "x": 0.74, "y": 0.57 },
                            { "x": 0.74, "y": 0.77 },
                            { "x": 0.81, "y": 0.77 }
                        ]}
                    ],
                    "mid": "left-back-wheel",
                    "type": "rectangle",
                    "categories": [{ "name": "WHEELS", "confidence": 100 }],
                }]
            },
            'RELATION_JOB': {
                'annotations': [
                    {
                        'categories': [{'name': 'WHOLE_CAR_AND_WHEELS', 'confidence': 100}],
                        'startObjects': [{'mid': 'car'}],
                        'endObjects': [{'mid': 'left-front-wheel'}, {'mid': 'left-back-wheel'}],
                    },
                ]
            },
        },
        'model_name': 'car-wheels-localisation-v0.0.1'
    }
)

#### - Polygon object


In [ ]:
project_examples.append(
    {
        'title': 'Car brand recognition - Polygon',
        'description': 'Identify and locate cars',
        'input_type': 'IMAGE',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "polygon"
                    ],
                    "instruction": "What car brand?",
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "TESLA": { "name": "Tesla", "children": [] },
                            "FERRARI": { "name": "Ferrari", "children": [] }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [
            "https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg", 
            "https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg"
        ],
        'json_response': {
            "JOB_0": {
                "annotations": [{
                    "boundingPoly": [{
                        "normalizedVertices": [
                            { "x": 0.16, "y": 0.52},
                            { "x": 0.16, "y": 0.76 },
                            { "x": 0.49, "y": 0.83 },
                            { "x": 0.82, "y": 0.76 },
                            { "x": 0.82, "y": 0.49 },
                            { "x": 0.70, "y": 0.32 },
                            { "x": 0.48, "y": 0.32 },
                        ]}
                    ],
                    "mid": "unique-tesla",
                    "type": "polygon",
                    "categories": [{ "name": "TESLA", "confidence": 100 }],
                }]
            }
        },
        'model_name': 'car-brand-localisation-v0.0.1'
    }
)

### c) Named Entities Recognition examples

#### - Simple text asset

In [3]:
project_examples.append(
    {
        'title': 'Grammatical categories',
        'description': 'Identify grammatical categories',
        'input_type': 'TEXT',
        'json_interface': {
            "jobs": {
                "JOB": {
                    "mlTask": "NAMED_ENTITIES_RECOGNITION",
                    "instruction": "Grammatical categories",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "NAME": { "name": "Name", "children": [] },
                            "NOUN": { "name": "Noun", "children": [] },
                            "ADJECTIVE": { "name": "Adjective", "children": [] },
                            "VERB": { "name": "Verb", "children": [] }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [
            "Chicago Bulls General Manager Jerry Krause was standing near a Gatorade cooler with medicine in hand as players and coaches milled around nearby.", 
            "Jordan, handsome and cool in a Nike sweatsuit, peered down at the hefty Krause, who was wearing a blue sweater tucked into high-rise pants."
        ],
        'json_response': {
            "JOB": {
                "annotations": [
                    {"categories": [{ "name": "NAME", "confidence": 100 }],
                    "beginOffset": 0,
                    "content": "Chicago Bulls",
                    "mid": "chicago"},
                    {"categories": [{ "name": "NAME", "confidence": 100 }],
                    "beginOffset": 30,
                    "content": "Jerry Krause",
                    "mid": "krause"},
                    {"categories": [{ "name": "NAME", "confidence": 100 }],
                    "beginOffset": 63,
                    "content": "Gatorade",
                    "mid": "gatorade"},
                    {"categories": [{ "name": "NOUN", "confidence": 100 }],
                    "beginOffset": 22,
                    "content": "Manager",
                    "mid": "manager"},
                    {"categories": [{ "name": "NOUN", "confidence": 100 }],
                    "beginOffset": 14,
                    "content": "General",
                    "mid": "general"},
                    {"categories": [{ "name": "NOUN", "confidence": 100 }],
                    "beginOffset": 84,
                    "content": "medicine",
                    "mid": "medicine"},
                    {"categories": [{ "name": "NOUN", "confidence": 100 }],
                    "beginOffset": 104,
                    "content": "players",
                    "mid": "players"},
                    {"categories": [{ "name": "NOUN", "confidence": 100 }],
                    "beginOffset": 116,
                    "content": "coaches",
                    "mid": "coaches"},
                    {"categories": [{ "name": "VERB", "confidence": 100 }],
                    "beginOffset": 124,
                    "content": "milled",
                    "mid": "milled"},
                    {"categories": [{ "name": "VERB", "confidence": 100 }],
                    "beginOffset": 43,
                    "content": "was standing",
                    "mid": "standing"},
                    {"categories": [{ "name": "NOUN", "confidence": 100 }],
                    "beginOffset": 96,
                    "content": "hand",
                    "mid": "hand"},
                    {"categories": [{ "name": "NOUN", "confidence": 100 }],
                    "beginOffset": 72,
                    "content": "cooler",
                    "mid": "cooler"}
                ]
            }
        },
        'model_name': 'ner-model-v0.0.1'
    }
)

#### - Rich text asset

In [ ]:
project_examples.append(
    {
        'title': 'Grammatical categories on law rich texts',
        'description': 'Identify grammatical categories',
        'input_type': 'TEXT',
        'json_interface':  {
            "jobs": {
                "JOB": {
                    "mlTask": "NAMED_ENTITIES_RECOGNITION",
                    "instruction": "Grammatical categories",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "NOMINAL_GROUP": { "name": "Nominal group", "children": [] },
                            "ADJECTIVE": {"name": "Adjective", "children": [] },
                            "VERB": {"name": "Verb", "children": [] }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import': [[
            {
                'children': [
                    {
                        'type': 'p',
                        'children': [
                            {
                                'id': 'PART1',
                                'bold': True,
                                'underline': True,
                                'text': 'The unanimous Declaration',
                            },
                            {
                                'id': 'PART2',
                                'bold': True,
                                'text': ' of the thirteen United States of America.',
                            },
                            {
                                'id': 'PART3',
                                'text':
                                "When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.",
                            }
                        ]
                    }
                ]
            }
        ]],
        'json_response': {
            "JOB": {
                "annotations": [
                    {
                        "beginId": 'PART1',
                        "beginOffset": 14,
                        "categories": [{ "name": "NOMINAL_GROUP", "confidence": 100 }],
                        "content": "Declaration of the thirteen United States of America",
                        "endId": 'PART2',
                        "endOffset": 41,
                        "mid": "declaration",
                    },
                ]
            }
        },
        'model_name': 'ner-model-v0.0.2'
    }
)

#### - Text asset with named-entities relations

In [ ]:
project_examples.append(
    {
        'title': 'Named entities relations',
        'description': 'Subject verb complement',
        'input_type': 'TEXT',
        'json_interface': {
            "jobs": {
                "NAMED_ENTITIES_RECOGNITION_JOB": {
                    "mlTask": "NAMED_ENTITIES_RECOGNITION",
                    "instruction": "Grammatical categories",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "SUBJECT": { "name": "Subject", "children": [] },
                            "VERB": { "name": "Verb", "children": [] },
                            "COMPLEMENT": { "name": "Complement", "children": [] }
                        },
                        "input": "radio"
                    },
                },
                "NAMED_ENTITIES_RELATION_JOB": {
                    "mlTask": "NAMED_ENTITIES_RELATION",
                    "instruction": "Grammatical relations",
                    "required": 0,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "VERB_AND_SUBJECT_S": {
                                "name": "Verb and subject(s)",
                                "children": [],
                                "startEntities": ["VERB"],
                                "endEntities": ["SUBJECT"]
                            },
                            "VERB_AND_COMPLEMENT_S": {
                                "name": "Verb and complement(s)",
                                "children": [],
                                "startEntities": ["VERB"],
                                "endEntities": ["COMPLEMENT"]
                            }
                        },
                        "input": "radio"
                    },
                }
            }
        },
        'assets_to_import': [
            "Jordan, handsome and cool in a Nike sweatsuit, peered down at the hefty Krause, who was wearing a blue sweater tucked into high-rise pants.",
            "Chicago Bulls General Manager Jerry Krause was standing near a Gatorade cooler with medicine in hand as players and coaches milled around nearby."
        ],
        'json_response': {
            "NAMED_ENTITIES_RECOGNITION_JOB": {
                "annotations": [
                    {"categories": [{ "name": "SUBJECT", "confidence": 100 }],
                    "beginOffset": 0,
                    "content": "Jordan",
                    "mid": "Jordan"},
                    {"categories": [{ "name": "VERB", "confidence": 100 }],
                    "beginOffset": 84,
                    "content": "was wearing",
                    "mid": "wearing"},
                    {"categories": [{ "name": "VERB", "confidence": 100 }],
                    "beginOffset": 111,
                    "content": "tucked",
                    "mid": "tucked verb"},
                    {"categories": [{ "name": "COMPLEMENT", "confidence": 100 }],
                    "beginOffset": 96,
                    "content": "a blue sweater tucked into high-rise pants",
                    "mid": "blue sweater complement"},
                    {"categories": [{ "name": "VERB", "confidence": 100 }],
                    "beginOffset": 47,
                    "content": "peered down",
                    "mid": "peered"},
                    {"categories": [{ "name": "COMPLEMENT", "confidence": 100 }],
                    "beginOffset": 62,
                    "content": "the hefty Krause",
                    "mid": "Krause complement"},
                    {"categories": [{ "name": "SUBJECT", "confidence": 100 }],
                    "beginOffset": 62,
                    "content": "the hefty Krause",
                    "mid": "Krause subject"}
                ]
            },
            "NAMED_ENTITIES_RELATION_JOB": {
                "annotations": [
                    {"categories": [{ "name": "VERB_AND_SUBJECT_S", "confidence": 100 }],
                    "startEntities": [{ "mid": "peered" }],
                    "endEntities": [{ "mid": "Jordan" }]},
                    {"categories": [{ "name": "VERB_AND_COMPLEMENT_S", "confidence": 100 }],
                    "startEntities": [{ "mid": "peered" }],
                    "endEntities": [{ "mid": "Krause complement" }]},
                    {"categories": [{ "name": "VERB_AND_SUBJECT_S", "confidence": 100 }],
                    "startEntities": [{ "mid": "wearing" }],
                    "endEntities": [{ "mid": "Krause subject" }]},
                    {"categories": [{ "name": "VERB_AND_COMPLEMENT_S", "confidence": 100 }],
                    "startEntities": [{ "mid": "wearing" }],
                    "endEntities": [{ "mid": "blue sweater complement" }]}
                ]
            }
        },
        'model_name': 'relation-model-v0.0.1'
    }
)

### d) Optical Character Recognition examples

#### - OCR (object detection + nested transcription)

In [ ]:
project_examples.append(
    {
        'title': 'OCR - ID cards',
        'description': 'Identify name and nationality',
        'input_type': 'IMAGE',
        'json_interface' : {
            "jobs": {
                "JOB_0": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "rectangle"
                    ],
                    "instruction": "Categories",
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "NATIONALITY": {
                                "name": "Nationality",
                                "children": [
                                    "JOB_1"
                                ],
                            },
                            "NAME": {
                                "name": "Name",
                                "children": [
                                    "JOB_2"
                                ],
                            }
                        },
                        "input": "radio"
                    }
                },
                "JOB_1": {
                    "mlTask": "TRANSCRIPTION",
                    "instruction": "Transcription of Nationality",
                    "required": 1,
                    "isChild": True,
                    "content": {
                        "input": "radio"
                    }
                },
                "JOB_2": {
                    "mlTask": "TRANSCRIPTION",
                    "instruction": "Transcription of Name",
                    "required": 1,
                    "isChild": True,
                    "content": {
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import' : [
            "https://upload.wikimedia.org/wikipedia/commons/3/3a/Dutch_identity_card_front_specimen_issued_9_March_2014.jpg",
        ],
        'json_response': {
            'JOB_0': {
                'annotations': [{
                    'boundingPoly': [{
                        'normalizedVertices': [
                            {'x': 0.47, 'y': 0.53},
                            {'x': 0.47, 'y': 0.48},
                            {'x': 0.65, 'y': 0.48},
                            {'x': 0.65, 'y': 0.53}
                        ]
                    }],
                    'categories': [{'name': 'NATIONALITY', 'confidence': 100}],
                    'mid': 'nationality',
                    'type': 'rectangle',
                    'children': {'JOB_1': {'text': 'Nederlandse'}}
                },
                {
                    'boundingPoly': [{
                        'normalizedVertices': [
                            {'x': 0.36, 'y': 0.37},
                            {'x': 0.36, 'y': 0.32},
                            {'x': 0.51, 'y': 0.32},
                            {'x': 0.51, 'y': 0.37}
                        ]
                    }],
                    'categories': [{'name': 'NAME', 'confidence': 100}],
                    'mid': 'name',
                    'type': 'rectangle',
                    'children': {'JOB_2': {'text': 'De Bruijn'}}
                }]
            }
        },
        'model_name' : 'transcription-model-v0.0.1'
    }
)

#### - OCR with relations

In [ ]:
project_examples.append(
    {
        'title': 'OCR - ID cards with relations',
        'description': 'Identify name, nationality and birth date and add relations',
        'input_type': 'IMAGE',
        'json_interface': {
            "jobs": {
                "JOB_0": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "rectangle"
                    ],
                    "instruction": "Categories",
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "NATIONALITY": {
                                "name": "Nationality",
                                "children": [
                                    "TRANSCRIPTION_JOB_1"
                                ],
                            },
                            "NAME": {
                                "name": "Name",
                                "children": [
                                    "TRANSCRIPTION_JOB_2"
                                ],
                            },
                            "BIRTH_DATE": {
                                "name": "Birth date",
                                "children": [
                                    "TRANSCRIPTION_JOB_3"
                                ],
                            }
                        },
                        "input": "radio"
                    }
                },
                "TRANSCRIPTION_JOB_1": {
                    "mlTask": "TRANSCRIPTION",
                    "instruction": "Transcription of Nationality",
                    "required": 1,
                    "isChild": True,
                    "content": {
                        "input": "radio"
                    }
                },
                "TRANSCRIPTION_JOB_2": {
                    "mlTask": "TRANSCRIPTION",
                    "instruction": "Transcription of Name",
                    "required": 1,
                    "isChild": True,
                    "content": {
                        "input": "radio"
                    }
                },
                "TRANSCRIPTION_JOB_3": {
                    "mlTask": "TRANSCRIPTION",
                    "instruction": "Transcription of Birth date",
                    "required": 1,
                    "isChild": True,
                    "content": {
                        "input": "radio"
                    }
                },
                "OCR_RELATION_JOB": {
                    "mlTask": "OBJECT_RELATION",
                    "content": {
                        "categories": {
                            "NAME_AND_NATIONALITY": {
                                "name": "Name and nationality",
                                "startObjects": ["NAME"],
                                "endObjects": ["NATIONALITY"]
                            },
                            "NAME_AND_BIRTH_DATE": {
                                "name": "Name and birth date",
                                "startObjects": ["NAME"],
                                "endObjects": ["BIRTH_DATE"]
                            }
                        },
                        "input": "radio"
                    },
                    "required": 0,
                    "isChild": False,
                    "instruction": "Relations"
                },
            },
        },
        'assets_to_import': [
            "https://upload.wikimedia.org/wikipedia/commons/3/3a/Dutch_identity_card_front_specimen_issued_9_March_2014.jpg",
        ],
        'json_response': {
            'JOB_0': {
                'annotations': [{
                    'boundingPoly': [{
                        'normalizedVertices': [
                            {'x': 0.47, 'y': 0.53},
                            {'x': 0.47, 'y': 0.48},
                            {'x': 0.65, 'y': 0.48},
                            {'x': 0.65, 'y': 0.53}
                        ]
                    }],
                    'categories': [{'name': 'NATIONALITY', 'confidence': 100}],
                    'type': 'rectangle',
                    'mid': 'netherlands',
                    'children': {'TRANSCRIPTION_JOB_1': {'text': 'Nederlandse'}}
                },
                {
                    'boundingPoly': [{
                        'normalizedVertices': [
                            {'x': 0.36, 'y': 0.37},
                            {'x': 0.36, 'y': 0.32},
                            {'x': 0.51, 'y': 0.32},
                            {'x': 0.51, 'y': 0.37}
                        ]
                    }],
                    'categories': [{'name': 'NAME', 'confidence': 100}],
                    'type': 'rectangle',
                    'mid': 'bruijn',
                    'children': {'TRANSCRIPTION_JOB_2': {'text': 'De Bruijn'}}
                },
                {
                    'boundingPoly': [{
                        'normalizedVertices': [
                            {'x': 0.36, 'y': 0.61},
                            {'x': 0.36, 'y': 0.56},
                            {'x': 0.63, 'y': 0.56},
                            {'x': 0.63, 'y': 0.61}
                        ]
                    }],
                    'categories': [{'name': 'BIRTH_DATE', 'confidence': 100}],
                    'type': 'rectangle',
                    'mid': 'date',
                    'children': {'TRANSCRIPTION_JOB_3': {'text': '10 MAA/MAR 1965'}}
                }]
            },
            "OCR_RELATION_JOB": {
                "annotations": [
                    {"categories": [{ "name": "NAME_AND_NATIONALITY", "confidence": 100 }],
                    "startObjects": [{ "mid": "bruijn" }],
                    "endObjects": [{ "mid": "netherlands" }]},
                    {"categories": [{ "name": "NAME_AND_BIRTH_DATE", "confidence": 100 }],
                    "startObjects": [{ "mid": "bruijn" }],
                    "endObjects": [{ "mid": "date" }]}
                ]
            }
        },
        'model_name': 'transcription-model-v0.0.1'
    }
)

### e) Speech-to-text example

In [ ]:
project_examples.append(
    {
        'title' : 'Kate Darling - TedX',
        'description' : 'Write down the speakers speeches',
        'input_type' : 'AUDIO',
        'json_interface' : {
            "jobs": {
                'SPEECH_TO_TEXT_JOB': {
                    "mlTask": "SPEECH_TO_TEXT",
                    "instruction": "",
                    "required": False,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "MUSIC": {
                                "name": "♫",
                                "children": []
                            },
                            "INAUDIBLE": {
                                "name": "[INAUDIBLE]",
                                "children": []
                            }
                        },
                        "input": "radio"
                    },
                },
                'diarization-job': {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "",
                    "required": False,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "SPEAKER 1": {
                                "name": "SPEAKER 1",
                                "children": []
                            },
                            "SPEAKER 2": {
                                "name": "SPEAKER 2",
                                "children": []
                            }
                        },
                        "input": "radio"
                    },
                  },
            }
        },
        'assets_to_import' : [
             'https://download.ted.com/talks/KateDarling_2018S-950k.mp4',
        ],
        'json_response' : {
            "SPEECH_TO_TEXT_JOB": {
                "annotations": [
                    {
                        "beginOffset": 13.02,
                        "categories": [
                            {
                                "name": "SPEAKER 1",
                                "confidence": 100
                            }
                        ],
                        "children": {
                            "speech_to_text_job": {
                                "annotations": [
                                  {
                                    "beginOffset": 13.02,
                                    "content": "There",
                                    "mid": "2020102114124590-18412-0"
                                  },
                                  {
                                    "beginOffset": 13.419285714285714,
                                    "content": "is",
                                    "mid": "2020102114124590-18412-1"
                                  },
                                  {
                                    "beginOffset": 13.818571428571428,
                                    "content": "a",
                                    "mid": "2020102114124590-18412-2"
                                  },
                                  {
                                    "beginOffset": 14.217857142857142,
                                    "content": "day.",
                                    "mid": "2020102114124590-18412-3"
                                  },
                                  {
                                    "beginOffset": 14.617142857142857,
                                    "content": "About",
                                    "mid": "2020102114124590-18412-4"
                                  },
                                  {
                                    "beginOffset": 15.016428571428571,
                                    "content": "ten",
                                    "mid": "2020102114124590-18412-5"
                                  },
                                  {
                                    "beginOffset": 15.415714285714285,
                                    "content": "years",
                                    "mid": "2020102114124590-18412-6"
                                  },
                                  {
                                    "beginOffset": 15.815,
                                    "content": "ago",
                                    "mid": "2020102114124590-18412-7"
                                  },
                                  {
                                    "beginOffset": 16.214285714285715,
                                    "content": "when",
                                    "mid": "2020102114124590-18412-8"
                                  },
                                  {
                                    "beginOffset": 16.61357142857143,
                                    "content": "I",
                                    "mid": "2020102114124590-18412-9"
                                  },
                                  {
                                    "beginOffset": 17.012857142857143,
                                    "content": "asked",
                                    "mid": "2020102114124590-18412-10"
                                  },
                                  {
                                    "beginOffset": 17.412142857142857,
                                    "content": "a",
                                    "mid": "2020102114124590-18412-11"
                                  },
                                  {
                                    "beginOffset": 17.81142857142857,
                                    "content": "friend",
                                    "mid": "2020102114124590-18412-12"
                                  },
                                  {
                                    "beginOffset": 18.210714285714285,
                                    "content": "to",
                                    "mid": "2020102114124590-18412-13"
                                  },
                                  {
                                    "beginOffset": 18.61,
                                    "content": "hold",
                                    "mid": "2020102114124590-18412-14"
                                  },
                                  {
                                    "beginOffset": 19.009285714285713,
                                    "content": "a",
                                    "mid": "2020102114124590-18412-15"
                                  },
                                  {
                                    "beginOffset": 19.408571428571427,
                                    "content": "baby",
                                    "mid": "2020102114124590-18412-16"
                                  },
                                  {
                                    "beginOffset": 19.80785714285714,
                                    "content": "dinosaur",
                                    "mid": "2020102114124590-18412-17"
                                  },
                                  {
                                    "beginOffset": 20.207142857142856,
                                    "content": "robot",
                                    "mid": "2020102114124590-18412-18"
                                  },
                                  {
                                    "beginOffset": 20.60642857142857,
                                    "content": "upside",
                                    "mid": "2020102114124590-18412-19"
                                  },
                                  {
                                    "beginOffset": 21.005714285714284,
                                    "content": "down.",
                                    "mid": "2020102114124590-18412-20"
                                  },
                                ]
                            }
                        },
                        "content": "There is a day. About ten years ago when I asked a friend to hold a baby dinosaur robot upside down.",
                        "mid": "2020102114124590-18412"
                    },
                    
                ]
            },
        },
        'model_name' : 'tedx-speech-v0.0.1'
    }
)

### f) PDF predictions

On PDF assets, you can use the same tasks as on text assets, which are:
- Classification (single-class, multi-class and nested),
- Named Entities Recognition,
- Transcription,
- Named entities relations.

All of these tasks have been introduced in previous steps.

However, you should notice that the form of the annotation for NER is different. Indeed, whereas an offset is used in texts, here the annotations work with the coordinates of the polygon the data belongs to and the page number.

In [ ]:
project_examples.append(
    {
        'title': 'Article titles and writers',
        'description': 'Identify the title and the writer of the article',
        'input_type': 'PDF',
        'json_interface' : {
            "jobs": {
                'NAMED_ENTITIES_RECOGNITION_JOB': {
                    "mlTask": "NAMED_ENTITIES_RECOGNITION",
                    "instruction": "Identify the ...",
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "TITLE": {
                                "name": "Title",
                                "children": [],
                            },
                            "WRITERS": {
                                "name": "Writers",
                                "children": [],
                            }
                        },
                        "input": "radio"
                    },

                },
            }
        },
        'assets_to_import': [
             'https://arxiv.org/pdf/1703.03365.pdf',
        ],
        'json_response': {
            'NAMED_ENTITIES_RECOGNITION_JOB': {
                'annotations': [{
                    'annotations': [{
                        'boundingPoly': [{
                            'normalizedVertices': [[
                                {'x': 0.28, 'y': 0.12},
                                {'x': 0.28, 'y': 0.15},
                                {'x': 0.72, 'y': 0.12},
                                {'x': 0.72, 'y': 0.15}
                            ]]
                        }],
                        'polys': [{
                            'normalizedVertices': [[
                                {'x': 0.28, 'y': 0.12},
                                {'x': 0.28, 'y': 0.15},
                                {'x': 0.72, 'y': 0.12},
                                {'x': 0.72, 'y': 0.15}
                            ]]
                        }],
                        'pageNumber': 1
                    }],
                    'categories': [{'name': 'TITLE', 'confidence': 100}],
                    'content': 'Learning Active Learning from Data',
                    'mid': 'article-title'
                }],
            }
        },
        'model_name': 'title-identify-v0.0.1'
    }
)

### g) Video frame projects

Frame projects are very similar to image ones as, in a frame project, each image is annotated independently.

As a result, the `jsonResponse` takes the shape of a dictionnary where each frame's annotations are linked to the index of this frame in the asset.
Here is an example with a 5 images Frame asset:

#### Frames classification

In [ ]:
project_examples.append(
    {
        'title' : 'Car classification video',
        'description' : 'Indicate if there is a vehicle close to the camera',
        'input_type' : 'FRAME',
        'json_interface' : {
            "jobs": {
                "FRAME_CLASSIFICATION_JOB": {
                    "mlTask": "CLASSIFICATION",
                    "instruction": "Is there at least one vehicle close to me?",
                    "required": 1,
                    "isChild": False,
                    "isVisible": True,
                    "content": {
                        "categories": {
                            "YES_THERE_IS": { "name": "Yes, there is", "children": [] },
                            "NO": { "name": "No", "children": [] }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import' : [
             ['https://storage.googleapis.com/label-public-staging/video1/video1-img000001.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000002.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000003.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000004.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000005.jpg']
        ],
        'json_response' : {
            "0": {
                "FRAME_CLASSIFICATION_JOB": {
                    'categories': [{'name': 'YES_THERE_IS', 'confidence': 100}],
                    'isKeyFrame': True
                }
            },
            "1": {
                "FRAME_CLASSIFICATION_JOB": {
                    'categories': [{'name': 'YES_THERE_IS', 'confidence': 100}],
                    'isKeyFrame': False
                }
            },
            "2": {
                "FRAME_CLASSIFICATION_JOB": {
                    'categories': [{'name': 'YES_THERE_IS', 'confidence': 100}],
                    'isKeyFrame': False
                }
            },
            "3": {
                "FRAME_CLASSIFICATION_JOB": {
                    'categories': [{'name': 'NO', 'confidence': 100}],
                    'isKeyFrame': True
                }
            },
            "4": {
                "FRAME_CLASSIFICATION_JOB": {
                    'categories': [{'name': 'NO', 'confidence': 100}],
                    'isKeyFrame': False
                }
            }
        },
        'model_name' : 'close-vehicle-identification-v0.0.1'
    }
)

#### Object detection (bounding boxes)

In [ ]:
project_examples.append(
    {
        'title' : 'Car tracking',
        'description' : 'Identify cars',
        'input_type' : 'FRAME',
        'json_interface' : {
            "jobs": {
                "JOB": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "rectangle"
                    ],
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "TESLA": {
                                "name": "Tesla",
                                "children": []
                            },
                            "FERRARI": {
                                "name": "Ferrari",
                                "children": []
                            }
                        },
                        "input": "radio"
                    }
                }
            }
        },
        'assets_to_import' : [
             ['https://storage.googleapis.com/label-public-staging/video1/video1-img000001.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000002.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000003.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000004.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000005.jpg']
        ],
        'json_response' : {
          "0": {
            "JOB": {
              "annotations": [
                {
                  "boundingPoly": [
                    {
                      "normalizedVertices": [
                        { "x": 0.57, "y": 0.59 },
                        { "x": 0.57, "y": 0.73 },
                        { "x": 0.68, "y": 0.73 },
                        { "x": 0.68, "y": 0.59 }
                      ]
                    }
                  ],
                  "categories": [{ "name": "TESLA", "confidence": 100 }],
                  "mid": "frame1-tesla",
                  "type": "rectangle",
                  "isKeyFrame": True
                }
              ]
            }
          },
          "1": {
            "JOB": {
              "annotations": [
                {
                  "boundingPoly": [
                    {
                      "normalizedVertices": [
                        { "x": 0.56, "y": 0.58 },
                        { "x": 0.56, "y": 0.74 },
                        { "x": 0.68, "y": 0.74 },
                        { "x": 0.68, "y": 0.58 }
                      ]
                    }
                  ],
                  "categories": [{ "name": "TESLA", "confidence": 100 }],
                  "mid": "frame2-tesla",
                  "type": "rectangle",
                  "isKeyFrame": False
                }
              ]
            }
          },
          "2": {
            "JOB": {
              "annotations": [
                {
                  "boundingPoly": [
                    {
                      "normalizedVertices": [
                        { "x": 0.55, "y": 0.58 },
                        { "x": 0.55, "y": 0.73 },
                        { "x": 0.67, "y": 0.73 },
                        { "x": 0.67, "y": 0.58 }
                      ]
                    }
                  ],
                  "categories": [{ "name": "TESLA", "confidence": 100 }],
                  "mid": "frame3-tesla",
                  "type": "rectangle",
                  "isKeyFrame": False
                }
              ]
            }
          },
          "3": {
            "JOB": {
              "annotations": [
                {
                  "boundingPoly": [
                    {
                      "normalizedVertices": [
                        { "x": 0.55, "y": 0.58 },
                        { "x": 0.55, "y": 0.72 },
                        { "x": 0.67, "y": 0.72 },
                        { "x": 0.67, "y": 0.58 }
                      ]
                    }
                  ],
                  "categories": [{ "name": "TESLA", "confidence": 100 }],
                  "mid": "frame4-tesla",
                  "type": "rectangle",
                  "isKeyFrame": False
                }
              ]
            }
          },
          "4": {
            "JOB": {
              "annotations": [
                {
                  "boundingPoly": [
                    {
                      "normalizedVertices": [
                        { "x": 0.54, "y": 0.58 },
                        { "x": 0.54, "y": 0.72 },
                        { "x": 0.66, "y": 0.72 },
                        { "x": 0.66, "y": 0.58 }
                      ]
                    }
                  ],
                  "categories": [{ "name": "TESLA", "confidence": 100 }],
                  "mid": "frame5-tesla",
                  "type": "rectangle",
                  "isKeyFrame": False
                }
              ]
            }
          }
        },
        'model_name' : 'car-tracker-v0.0.1'
    }
)

#### Object detection with nested classification

In [ ]:
project_examples.append(
    {
        'title' : 'Car tracking + Nested',
        'description' : 'Identify cars',
        'input_type' : 'FRAME',
        'json_interface' : {
            "jobs": {
                "DETECTION_JOB": {
                    "mlTask": "OBJECT_DETECTION",
                    "tools": [
                        "rectangle"
                    ],
                    "required": 1,
                    "isChild": False,
                    "content": {
                        "categories": {
                            "TESLA": {
                                "name": "Tesla",
                                "children": ["CLASSIFICATION_JOB_1"]
                            },
                            "FERRARI": {
                                "name": "Ferrari",
                                "children": ["CLASSIFICATION_JOB_2"]
                            }
                        },
                        "input": "radio"
                    }
                },
                "CLASSIFICATION_JOB_1": {
                    "mlTask": "CLASSIFICATION",
                    "isChild": True,
                    "content": {
                        "categories": {
                            "IS_THE_WHOLE_CAR_VISIBLE": {
                                "name": "Is the whole car visible?",
                                "children": []
                            }
                        },
                        "input": "checkbox"
                    }
                },
                "CLASSIFICATION_JOB_2": {
                    "mlTask": "CLASSIFICATION",
                    "isChild": True,
                    "content": {
                        "categories": {
                            "IS_THE_WHOLE_CAR_VISIBLE": {
                                "name": "Is the whole car visible?",
                                "children": []
                            }
                        },
                        "input": "checkbox"
                    }
                }
            }
        },
        'assets_to_import' : [
             ['https://storage.googleapis.com/label-public-staging/video1/video1-img000001.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000002.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000003.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000004.jpg',
              'https://storage.googleapis.com/label-public-staging/video1/video1-img000005.jpg']
        ],
        'json_response' : {
            "0": {
                "DETECTION_JOB": {
                    "annotations": [{
                        "boundingPoly": [{
                            "normalizedVertices": [
                                { "x": 0.57, "y": 0.59 },
                                { "x": 0.57, "y": 0.73 },
                                { "x": 0.68, "y": 0.73 },
                                { "x": 0.68, "y": 0.59 }
                            ]
                        }],
                        "categories": [{ "name": "TESLA", "confidence": 100 }],
                        "children": {
                            "CLASSIFICATION_JOB_1": {
                                "categories": [{ "name": "IS_THE_WHOLE_CAR_VISIBLE", "confidence": 100 }],
                                'isKeyFrame': True
                            }
                        },
                        "mid": "frame1-tesla",
                        "type": "rectangle",
                        "isKeyFrame": True
                    }]
                },
            },
            "1": {
                "DETECTION_JOB": {
                    "annotations": [{
                        "boundingPoly": [{
                            "normalizedVertices": [
                                { "x": 0.56, "y": 0.58 },
                                { "x": 0.56, "y": 0.74 },
                                { "x": 0.68, "y": 0.74 },
                                { "x": 0.68, "y": 0.58 }
                            ]
                        }],
                        "categories": [{ "name": "TESLA", "confidence": 100 }],
                        "children": {
                            "CLASSIFICATION_JOB_1": {
                                "categories": [{ "name": "IS_THE_WHOLE_CAR_VISIBLE", "confidence": 100 }],
                                'isKeyFrame': False
                            }
                        },
                        "mid": "frame2-tesla",
                        "type": "rectangle",
                        "isKeyFrame": False
                    }]
                }
            },
            "2": {
                "DETECTION_JOB": {
                    "annotations": [{
                        "boundingPoly": [{
                            "normalizedVertices": [
                                { "x": 0.55, "y": 0.58 },
                                { "x": 0.55, "y": 0.73 },
                                { "x": 0.67, "y": 0.73 },
                                { "x": 0.67, "y": 0.58 }
                            ]
                        }],
                        "categories": [{ "name": "TESLA", "confidence": 100 }],
                        "children": {
                            "CLASSIFICATION_JOB_1": {
                                "categories": [{ "name": "IS_THE_WHOLE_CAR_VISIBLE", "confidence": 100 }],
                                'isKeyFrame': False
                            }
                        },
                        "mid": "frame3-tesla",
                        "type": "rectangle",
                        "isKeyFrame": False
                    }]
                }
            },
            "3": {
                "DETECTION_JOB": {
                    "annotations": [{
                        "boundingPoly": [{
                            "normalizedVertices": [
                                { "x": 0.55, "y": 0.58 },
                                { "x": 0.55, "y": 0.72 },
                                { "x": 0.67, "y": 0.72 },
                                { "x": 0.67, "y": 0.58 }
                            ]
                        }],
                        "categories": [{ "name": "TESLA", "confidence": 100 }],
                        "children": {
                            "CLASSIFICATION_JOB_1": {
                                "categories": [{ "name": "IS_THE_WHOLE_CAR_VISIBLE", "confidence": 100 }],
                                'isKeyFrame': False
                            }
                        },
                        "mid": "frame4-tesla",
                        "type": "rectangle",
                        "isKeyFrame": False
                    }]
                }
            },
            "4": {
                "DETECTION_JOB": {
                    "annotations": [{
                        "boundingPoly": [{
                            "normalizedVertices": [
                                { "x": 0.54, "y": 0.58 },
                                { "x": 0.54, "y": 0.72 },
                                { "x": 0.66, "y": 0.72 },
                                { "x": 0.66, "y": 0.58 }
                            ]
                        }],
                        "categories": [{ "name": "TESLA", "confidence": 100 }],
                        "children": {
                            "CLASSIFICATION_JOB_1": {
                                "categories": [{ "name": "IS_THE_WHOLE_CAR_VISIBLE", "confidence": 100 }],
                                'isKeyFrame': False
                            }
                        },
                        "mid": "frame5-tesla",
                        "type": "rectangle",
                        "isKeyFrame": False
                    }]
                }
            }
        },
        'model_name' : 'car-tracker-v0.0.2'
    }
)

Finally, with the following code snipet and the various examples above, you can set up an example for each project live on your account Kili:

In [ ]:
import time

for project_example in project_examples:
    project = kili.create_project(title=project_example['title'],
                                  description=project_example['description'],
                                  input_type=project_example['input_type'],
                                  json_interface=project_example['json_interface'])
    
    assets_to_import = project_example['assets_to_import']
    if len(assets_to_import) == 0:
        continue
    if isinstance(assets_to_import[0], dict) or isinstance(assets_to_import[0], list):
        content_array = ['']*len(assets_to_import)
        json_content_array = assets_to_import
    else:
        content_array = assets_to_import
        json_content_array = None
    kili.append_many_to_dataset(
        project_id=project['id'], 
        content_array=content_array,
        external_id_array=['ex1','ex2'],
        json_content_array=json_content_array)
    
    id_check = kili.create_predictions(
        project_id=project['id'],
        external_id_array=['ex1'],
        model_name_array=[project_example['model_name']],
        json_response_array=[project_example['json_response']])
    
    assert(id_check['id']== project['id']) # just a check to assert that everything is running


## 4. Visualizing predictions in Kili
To verify that the prediction in question was indeed pushed in Kili, you can go to https://cloud.kili-technology.com/label/projects/[PROJECT_ID]/dataset/?view=table&currentPage=1&pageSize=20. You should get a new `PREDICTION` line like this:

![update settings](img/import_predictions-visualize_prediction.png)

## Summary
In this tutorial, we accomplished the following:
- We introduced the concept of Kili interface settings and the 4 different kind of labels (`DEFAULT`, `PREDICTION`, `AUTOSAVE`, `REVIEW`).
- We demonstrated how to retrieve a `DEFAULT` label.
- We discussed all the possible `PREDICTION` labels that can be imported on the platform.
 
If you enjoyed this tutorial, check out the other Recipes for other tutorials that you may find interesting, including demonstrations of how to use Kili.

You can also visit the Kili website or Kili documentation for more info!

